In [24]:
import pandas as pd
import numpy as np
import re

import re
import jellyfish
import pymorphy2

# visualisation libraries
import matplotlib.pyplot as plt
import seaborn as sns
large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': small,
          'figure.figsize': (14, 9),
          'axes.labelsize': small,
          'axes.titlesize': small,
          'xtick.labelsize': small,
          'ytick.labelsize': small,
          'figure.titlesize': large}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")

%matplotlib inline

In [36]:
# Загружаем БД:
df = pd.read_excel('data/data.xlsx')

In [37]:
# Меняем названия на более приемлемые:

target_shops = [
    'АО "ТОРГОВЫЙ ДОМ "ПЕРЕКРЕСТОК"', 'ООО "АГРОТОРГ"', 'ООО "ЛЕНТА"',
   'АО "ДИКСИ ЮГ"', 'ООО "АШАН"', 'АО "ТАНДЕР"', 'ООО "БИЛЛА"', 'ООО "О`КЕЙ"',
    'ООО "АГРОАСПЕКТ"', 'ООО "АТАК"'
]

shops_rename = {
    'АО "ТОРГОВЫЙ ДОМ "ПЕРЕКРЕСТОК"': 'ПЕРЕКРЕСТОК',
    'ООО "АГРОТОРГ"': 'ПЯТЕРОЧКА',
    'ООО "ЛЕНТА"': 'ЛЕНТА',
    'АО "ДИКСИ ЮГ"': 'ДИКСИ',
    'ООО "АШАН"': 'АШАН',
    'АО "ТАНДЕР"': 'МАГНИТ',
    'ООО "БИЛЛА"': 'БИЛЛА',
    'ООО "О`КЕЙ"': 'О`КЕЙ',
    'ООО "АГРОАСПЕКТ"': 'ПЯТЕРОЧКА',
    'ООО "АТАК"': 'АТАК'
}

df = df[df['shop_name'].isin(target_shops)].copy()
df['shop_name'] = df['shop_name'].apply(lambda x: shops_rename[x])
df.sample()

,shop_name,name,quantity,price,sum
44855,АШАН,"ЧАЙ ЧЕРН,ШАХ ГОЛД",1.0,219.99,219.99


In [38]:
# Выбираем только магазин ПЯТЕРОЧКА:
df = df[(df['shop_name'] == 'ПЕРЕКРЕСТОК') | (df['shop_name'] == 'ПЯТЕРОЧКА')].drop_duplicates('name').reset_index(drop=True)
df.head()

,shop_name,name,quantity,price,sum
0,ПЕРЕКРЕСТОК,1:3182857 Яйца куриные столовые 10шт,3.0,55.9,167.7
1,ПЕРЕКРЕСТОК,2*3669625 Кондиционер LENOR 910мл,1.0,119.9,119.9
2,ПЕРЕКРЕСТОК,3:3500750 Средство FAIRY 450мл,1.0,94.9,94.9
3,ПЕРЕКРЕСТОК,4*3281576 Порошок DOSIA COLOR 400г,1.0,39.9,39.9
4,ПЕРЕКРЕСТОК,5*70709 БЗМЖ Молоко PARMALAT 1л,2.0,59.9,119.8


In [39]:
# Смотрим, в каком виде существуют данные:
print('\n'.join(df['name'].sample(20).values))

2*3417343 Блинчики В ВЕЛИКИЙ ПОСТ 420г
19*3680039 Печенье ЛЮБЯТОВО 280г
6*3671632 Форель НОВЫЙ ОКЕАН 200г
16*: 3953339 КИПР.Сыр ШВЕЙЦАРСКИЙ 50% 25
3070382 ХРУСТИМ Сухарики СМЕТАНА 40г
14*3633180 Томаты ЧЕРРИ 500г
40*3611009 Чипсы NATURALS 100г
24: 3433428 БЗМЖ Кефир АСЕНЬЕВ.ФЕРМА 450мл
3496904 KIT.Корм куриц.соус.д/взр.85г
3610732 КОР.ИЗ КОР.Варенец терм.4% 350г
2173061 БЗМЖ Масло КОР.ИЗ КОРЕНОВ. 180г
14:3468954 БЗМЖ Масло КРЕСТЬЯН.УЗОРЫ 400г
10*: 31895 Салат лист.горш.1шт РОССИЯ
3014166 Пельмени Останкино Традиц.0,5кг
9:83808 Грудка ПЕТЕЛИНО куриная 1кг
4012150 ASSAND Чай МОЛОЧ.УЛУН пак.25х2г
21:3182853 Яйца куриные 10шт
9*3008911 Окорок РОССИЙСКИЙ 150г
3195121 Вод.ЕССЕН.цел.мин.ск№73ст/б0,5л
2059866 Газета АНТЕННА мини А4


#### Избавляемся от цифр в начале:

In [56]:
def remove_digits(name: str)-> str:
    """Удаляет цифры в начале."""

    result = []
    name = name.split()[1:]
    if name:
        if name[0].isdigit():
            name = name[1:]
    return ' '.join(name)

In [57]:
df['clear_name'] = df['name'].apply(remove_digits)
df.sample(10)

,shop_name,name,quantity,price,sum,clear_name,size
5810,ПЕРЕКРЕСТОК,3*3639244 Колбаса ДОКТОРСКАЯ 400г,1.000,149.90,149.90,Колбаса ДОКТОРСКАЯ 400г,400г
5323,ПЕРЕКРЕСТОК,"10*3659093 Напиток NEMOLOKO 0,25л",4.000,29.90,119.60,"Напиток NEMOLOKO 0,25л","0,25л"
7968,ПЕРЕКРЕСТОК,4017449 Хлеб ЛИТОВСКИЙ МИЛИМАС 300г,1.000,43.81,43.81,Хлеб ЛИТОВСКИЙ МИЛИМАС 300г,300г
11302,ПЕРЕКРЕСТОК,16: 3673236 Филе бедра ИНДИЛАЙТ 410г,1.000,219.00,219.00,Филе бедра ИНДИЛАЙТ 410г,410г
7927,ПЕРЕКРЕСТОК,2*3455601 БЗМЖ Сыр НАТ.ПРОД.СУЛУГУНИ 330г,1.000,189.90,189.90,БЗМЖ Сыр НАТ.ПРОД.СУЛУГУНИ 330г,330г
15772,ПЯТЕРОЧКА,3327002 BOD.Мыло СВЕЖ.ЦИТРУС жид.500мл,1.000,75.49,75.49,BOD.Мыло СВЕЖ.ЦИТРУС жид.500мл,500мл
14903,ПЯТЕРОЧКА,*3934850 К.Ц.Яйца кур.стол.С0 10шт,1.000,64.99,64.99,К.Ц.Яйца кур.стол.С0 10шт,10шт
12701,ПЯТЕРОЧКА,"3951290 FRESH Нап.SPRITZ APER.б/а 0,5л",1.000,29.99,29.99,"FRESH Нап.SPRITZ APER.б/а 0,5л","0,5л"
11336,ПЕРЕКРЕСТОК,10: 3388814 БЗМЖ Сыр БРЕСТ-ЛИТОВСКИЙ 150г,1.000,119.90,119.90,БЗМЖ Сыр БРЕСТ-ЛИТОВСКИЙ 150г,150г
5418,ПЕРЕКРЕСТОК,12:1532 Киви 1кг,1.058,129.90,137.43,Киви 1кг,1кг


#### Работаем с весом, объёмом товара и количества:

In [58]:
# Находим вес или объём товара:

def find_weight_or_volume(name: str)-> str:
    """Находит вес или объём товара."""

    pattern1 = r'\d+((\.|\,|x|X|х|Х)?)\d*([а-я]|[А-Я])*$' # ищет вес в конце
    pattern2 = r'\d+\w+' #  ищет вес в середине
    pattern3 = r'(\d+ кг)|(\d+ г)'  # находит `1 кг` или `1 г`
    result = re.search(pattern1, name)
    if result:
        return result.group(0)
    result = re.search(pattern2, name)
    if result:
        return result.group(0)
    result = re.search(pattern3, name)
    return result.group(0) if result else ''
    

df['size'] = df['clear_name'].apply(find_weight_or_volume)
df.head()

,shop_name,name,quantity,price,sum,clear_name,size
0,ПЕРЕКРЕСТОК,1:3182857 Яйца куриные столовые 10шт,3.0,55.9,167.7,Яйца куриные столовые 10шт,10шт
1,ПЕРЕКРЕСТОК,2*3669625 Кондиционер LENOR 910мл,1.0,119.9,119.9,Кондиционер LENOR 910мл,910мл
2,ПЕРЕКРЕСТОК,3:3500750 Средство FAIRY 450мл,1.0,94.9,94.9,Средство FAIRY 450мл,450мл
3,ПЕРЕКРЕСТОК,4*3281576 Порошок DOSIA COLOR 400г,1.0,39.9,39.9,Порошок DOSIA COLOR 400г,400г
4,ПЕРЕКРЕСТОК,5*70709 БЗМЖ Молоко PARMALAT 1л,2.0,59.9,119.8,БЗМЖ Молоко PARMALAT 1л,1л


In [321]:
# Смотрим, какие записи не получилось распознать:
print('\n'.join(df[df['size'] == '']['name'].values))

Скретч-карта THE VOICE
Скретч-карта THE VOICE
Пакет ПЕРЕКРЕСТОК
Напиток кокосовый с рисом
ПЕРЕКРЕСТОК Пакет майка
GREENF.Чай ГОЛДЕН ЦЕЙЛ.чер.
VILEDA Губка ТИП-ТОП классич
СПз Круассан клас.на масле
KUCH.Кекс ВЕСЕННИЙ ШТОЛЛЕН
Пакет ПЕРЕКРЕСТОК майка
Набор BAFFY MAGIC TIME
Скретч-карта THE VOICE
СИГАРЕТЫ VOGUE BLEUE
Скретч-карта THE VOICE
Пасочница ДОМАШНЯЯ КУХНЯ
Томаты розовые Азербайджан
THE VOICE Скретч-карта ГОЛО
Пакет ПЕРЕКРЕСТОК майка
Пакет ПЕРЕКРЕСТОК майка
Сырок творожный ВкусВилл со сгущенным
Доставка
Бумага КРУГЛЫЙ ГОД 8 рулонов
Бумага КРУГЛЫЙ ГОД туалетная
Губка PACLAN PRACTI MAGIC
Перчатки HOME STORY рS
Наклейка Голубая
Скретч-карта THE VOICE
ПЕРЕКРЕСТОК Пакет майка
ЗД.МЕН.Нап.МОЛ.СОЕВ.раст.сыр
ЗД.МЕН.Нап.МОЛ.СОЕВ.раст.сыр
Пакет ПЕРЕКРЕСТОК майка
Смеси ДОМАШНЯЯ КУХНЯ д/окрашив
Скретч-карта THE VOICE
Скретч-карта THE VOICE
Скретч-карта THE VOICE
СИГАРЕТЫ WINSTON XS BLUE
СИГАРЕТЫ WINSTON COMPACT
Скретч-карта THE VOICE
Творог ВкусВилл Шоколадная крошка мя
Фарш ВкусВилл Домашний 

##### Вроде бы получилось распознать почти всё

In [66]:
def remove_size_from_name(clear_name: str, size: str)-> str:
    '''Удаляет из названия товара цену.'''

    return clear_name.replace(size, '').strip().strip('.')

df[['clear_name']] = df[['clear_name', 'size']].apply(lambda x: remove_size_from_name(**x), axis=1)
df.head()

,shop_name,name,quantity,price,sum,clear_name,size
0,ПЕРЕКРЕСТОК,1:3182857 Яйца куриные столовые 10шт,3.0,55.9,167.7,Яйца куриные столовые,10шт
1,ПЕРЕКРЕСТОК,2*3669625 Кондиционер LENOR 910мл,1.0,119.9,119.9,Кондиционер LENOR,910мл
2,ПЕРЕКРЕСТОК,3:3500750 Средство FAIRY 450мл,1.0,94.9,94.9,Средство FAIRY,450мл
3,ПЕРЕКРЕСТОК,4*3281576 Порошок DOSIA COLOR 400г,1.0,39.9,39.9,Порошок DOSIA COLOR,400г
4,ПЕРЕКРЕСТОК,5*70709 БЗМЖ Молоко PARMALAT 1л,2.0,59.9,119.8,БЗМЖ Молоко PARMALAT,1л


### Находим производителя:

In [67]:
brands = pd.read_csv('data/brands.csv')['0']

In [68]:
black_list = [
    'ВАНИЛИН', 'СИГАРЕТЫ', 'НАРЕЗНОЙ', 'МАНГО', 'АПЕЛ', 'МОЛОЧНЫЕ', 'МОЛОЧНЫЕ', 'КУКР', 'ST', 'COMP', 'BLUE',
    'ДОМ', 'ХЛОП', 'МАРИЯ', 'ТРАДИЦИОННОЕ', 'ULTRA', 'ЛЕС', 'ЯГ', 'МЕДВЕЖ', 'КОЛЬЦА', 'ВАН', 'СТР', 'РЕЦ',
    'ВЫГОДНО', 'СЛИВОЧНЫЙ', 'ГЕРКУЛЕС', 'ULTRA', 'SUPER', 'ИТАЛ', 'ГРЕЧЕСК', 'БЗМЖ', 'C', 'L', 'Б', 'С',
    'КРАБ', 'ДЕРЕВЕНСК', 'C1', 'MINI', 'КРАБОВЫЙ', 'КАРБОНАРА', 'СОСИСКА', 'РУ', 'ГРЕЧЕСК', 'ХЛЕБ',
    'С', 'ОТРУБЯМИ','АРМЯНСКИЙ',  'ИЗЮМ', 'КИШМИШ', 'ВЕНГЕРСКАЯ', 'ПОСЫПКА', 'ЦВ', 'ШАР', 'СВЕТ',
    'ЛИМ', 'ВКУС', 'ВОЗДУШНЫЙ', 'ПЭТ', 'ГРАНАТОВЫЙ', 'ЩЕЛКОВСКИЙ', 'БЗМЖ'
]

def find_brand(name: str)-> str:
    """Находит бренд из слов в верхнем регистре."""
    
    pattern = r'(\b[А-Я]+[0-9]*\b|\b[A-Z]+[0-9]*\b)' # Ищет слова в верхнем регистре
    result = re.findall(pattern, name)
    
    [result.remove(x) for x in result if x in black_list] # удаляю мусор из найденных брендов
    result = [x.lower() for x in result]
    if not result:
        return 'None'
    result_str = ' '.join(result)
    similar_brands = brands[brands.apply(lambda x: x.startswith(result[0].lower()))].values
    dist = [jellyfish.jaro_distance(result_str, x) for x in similar_brands]
    if not dist:
        return 'None'
            
    return similar_brands[np.argmax(dist)]

In [69]:
df['brand'] = df['clear_name'].apply(find_brand)
df.sample(10)

,shop_name,name,quantity,price,sum,clear_name,size,brand
5952,ПЕРЕКРЕСТОК,2:3182857 Яйца куриные столовые 10шт,1.0,57.90,57.90,Яйца куриные столовые,10шт,None
7469,ПЕРЕКРЕСТОК,3: 3422668 Рис АГРО-АЛЬЯНС 900г,1.0,94.90,94.90,Рис АГРО-АЛЬЯНС,900г,агроальянс
491,ПЕРЕКРЕСТОК,11: 82760 ШЕБЕК.Макар.СПАГЕТ.тонк.450г,1.0,55.90,55.90,ШЕБЕК.Макар.СПАГЕТ.тонк,450г,шебекинские
10040,ПЕРЕКРЕСТОК,1:2149542 Батон НАРЕЗНОЙ 400г,1.0,43.90,43.90,Батон НАРЕЗНОЙ,400г,None
4049,ПЕРЕКРЕСТОК,"*3919811 Сувенир ЦЫПЛЯТА 4,5см",1.0,129.90,129.90,Сувенир ЦЫПЛЯТА,"4,5см",None
10758,ПЕРЕКРЕСТОК,6*3405386 Брокколи МОРОЗКО GREEN 400г,1.0,99.90,99.90,Брокколи МОРОЗКО GREEN,400г,морозко green
14122,ПЯТЕРОЧКА,3470342 Журнал Бабушкин пирог,1.0,63.99,63.99,Журнал Бабушкин пирог,,None
10203,ПЕРЕКРЕСТОК,1:3648240 БЗМЖ Сыр РОССИЙСКИЙ 200г,1.0,129.00,129.00,БЗМЖ Сыр РОССИЙСКИЙ,200г,российский
2165,ПЕРЕКРЕСТОК,7*3417345 Блинчики В ВЕЛИКИЙ ПОСТ 420г,1.0,75.90,75.90,Блинчики В ВЕЛИКИЙ ПОСТ,420г,в великий пост
7123,ПЕРЕКРЕСТОК,3016763 Яблоки МИКС фасованные 1кг,1.5,79.99,119.99,Яблоки МИКС фасованные,1кг,миксэль


In [70]:
def remove_brand_from_name(name: str, brand: str)-> str:
    """Удаляет бренды, которые получилось найти из названия товара."""

    if brand == 'None':
        return name.replace('.', ' ').lower()

    result = []
    for word in name.replace('.', ' ').lower().split():
        if brand.startswith(word):
            continue
        result.append(word)
    return ' '.join(result)

In [72]:
df['clear_name'] = df[['clear_name', 'brand']].apply(lambda x: remove_brand_from_name(x['clear_name'], x['brand']), axis=1)

In [74]:
df['clear_name'].sample(10)

12208           молоко сгущ цел 8,5%
795      пюре фруто-няня яблоч-банан
1449                           кумин
8273         пряники полет  заварные
6722                огурцы мар целые
2942                        скумбрия
10809                         лимоны
233                      вода родник
10816             мандарины с листом
12                 вермишель длинная
Name: clear_name, dtype: object

### Работаем с наименованием продука

In [75]:
PRODUCT_DICT = { # 2530 слов можжно заменить этим словарём
    'марин': 'маринованные',
    'морож': "мороженное",
    'плом': 'пломбир',
    'ван': 'ванильный',
    'сл': 'сливочное(слоеное)',
    'ароматиз': 'ароматизатор',
    'пищ': 'пищевой',
    'нап': 'напиток',
    'напи': 'напиток',
    'напит': 'напиток',
    'пив': 'пиво',
    'сух': 'сухой(сухарь)',
    'изм': 'измельчённый',
    'пельм': 'пельмени',
    'завтр': 'завтрак',
    'молшок': 'молочный шоколад',
    'бискв': 'бисквит',
    'глад': 'гладкие',
    'хлоп': 'хлопья',
    'конф': 'конфеты',
    'шокол': 'шоколад',
    'слив': 'сливочное',
    'сгущ': 'сгущеное',
    'конд': 'кондитерская',
    'слад': 'сладкая',
    'лес': 'лесные',
    'яг': 'ягоды',
    'медвеж': 'медвежье',
    'эрив': 'эривань',
    'национал': 'национальный',
    'карт': 'картотфель',
    'хруст': 'хрустящий',
    'мин': 'минеральная',
    'пит': 'питьевая',
    'мор': 'морская',
    'среднепл': 'среднеплодные',
    'майон': 'майонез',
    'мак': 'макароны',
    'кур': 'курица',
    'лап': 'лапша',
    'печ': 'печенье',
    'салф': 'салфетки',
    'нарез': 'нарезной',
    'бум': 'бумага',
    'бел': 'белый(ая)',
    'прян': 'пряники',
    'классич': 'классический',
    'пломб': 'пломбир',
    'обезж': 'обезжиренный',
    'итал': 'итальянский',
    'свет': 'светлое',
    'шок': 'шоколад(ный)',
    'московск': 'московский(ая)',
    'проклад': 'прокладка',
    'кукур': 'кукуруза(ой)',
    'бумаж': 'бумажое',
    'деревенск': 'деревенское(ая)',
    'клуб': 'клубника',
    'абр': 'абрикос',
    'виш': 'вишня(ей)',
    'охл': 'охлаждённое(ые)',
    'йог': 'йогурт',
    'карам': 'карамель(ю)',
    'черн': 'черника(черникой)',
    'соев': 'соевый',
    'газ': 'газированный',
    'туал': 'туалетный(ая)',
    'зер': 'зерновой(ая)',
    'зерн': 'зерновой(ая)',
    'слоен': 'слоеное',
    'ябл': 'яблоко',
    'перс': 'персик',
    'груд': 'грудка',
    'охл': 'охлаждённая',
    'тон': 'тонкий(ая)',
    'упаков': 'упаковка(е)',
    'цв': 'цветной(ых)',
    'подл': 'подложка(е)',
    'батонч': 'батончик',
    'бат': 'батон(чик)',
    'шоколадн': 'шоколадный',
    'глаз': 'глазурь',
    'апел': 'апельсин',
    'теляч': 'телятина',
    'древ': 'древесный',
    'туал': 'туалет',
    'подс': 'подсолнечное',
    'кур': 'куриный(ая)',
    'свин': 'свинина',
    'круас': 'круасаны',
    'дет': 'детский',
    'базил': 'базилик',
    'овс': 'овсяные',
    'пос': 'посуда',
    'нат': 'натуральный(ая)',
    'обжар': 'обжаренные',
    'трюф': 'трюфель',
    'элит': 'элитный',
    'проклад': 'прокладка',
    'майон': 'майонез',
    'сгущ': 'сгущёнка',
    'вар': 'вареная(ый)',
    'готов': 'готовый(ая)',
    'жев': 'жевательная',
    'рез': 'резинка',
    'рж': 'ржаная(ой)',
    'лик': 'ликер',
    'бифшт': 'бифштекс',
    'сосис': 'сосиски',
    'твор': 'творог(твороженный)',
    'ассорт': 'ассортимент(е)',
    'бискв': 'бисквит',
    'биск': 'бисквит',
    'лаком': 'лакомство',
    'класс': 'классическое(ий)',
    'прем' : 'премиум',
    'пшенич' : 'пшеничный(ая)',
    'кваш' : 'квашенный(ая)',
    'кетч' : 'кетчуп',
    'пирож': 'пироженное',
    'подуш': 'подушечки',
    'нач': 'начинка',
    'изд': 'изделие(я)',
    'пюр': 'пюре',
    'темн': 'темный',
    'колб': 'колбаса',
    'наб': 'набор',
    'яблочн': 'яблочный',
    'освет': 'осветлённый',
    'tea': 'чай',
    'шамп': 'шампунь',
    'осв': 'освежитель',
    'воз': 'воздух(а)',
    'возд': 'воздух(а)'
}

In [76]:
def remove_one_char(name: str) -> str:
    """Удаляет слово из одной буквы."""

    name = name.split()
    words = []
    for word in name:
        if len(word) == 1 and word not in ['в', 'c']:
            continue
        words.append(word)
    return ' '.join(words)

In [77]:
df['clear_name'] = df['clear_name'].apply(remove_one_char)

In [78]:
def remove_two_chars(name: str)-> str:
    """Удаляет слова, состоящие из двух букв."""

    name = name.split()
    words = []
    for word in name:
        if len(word) == 2 and '%' not in word and word not in ['из']: 
            continue
        words.append(word)
    return ' '.join(words)

In [79]:
df['clear_name'] = df['clear_name'].apply(remove_two_chars)

In [80]:
def use_product_dict(name):
    """Использут словарь для дополнения слов."""

    words = []
    name = name.split()
    for word in name:
        words.append(PRODUCT_DICT.get(word, word))
    return ' '.join(words)

In [81]:
df['clear_name'] = df['clear_name'].apply(use_product_dict)

In [82]:
def delete_three_english_chars(name: str)-> str:
    """Удаляет мусорный слова их 3-ёх букв на английском."""

    words = []
    pattern = r'[a-z]+'
    name = name.split()
    for word in name:
        if len(word) <= 3 and re.findall(pattern, word) and word not in ['xxl', 'old', 'm&m', 'plu']:
            continue
        words.append(word)
    return ' '.join(words)

In [83]:
df['clear_name'] = df['clear_name'].apply(delete_three_english_chars)

In [84]:
def normalize_name(name: str, morph: pymorphy2.MorphAnalyzer)-> str:
    """Использует pymorphy2 для нормализаци слов."""
    
    result = [morph.parse(word)[0].normal_form for word in s.split()]
    return ' '.join(result)

In [85]:
# Не зашло, лучше не использовать
# Также не особо зашло https://github.com/natasha/natasha
# morph = pymorphy2.MorphAnalyzer()
# df['clear_name'] = df['clear_name'].apply(lambda x: normalize_name(x, morph))

In [86]:
df.sample(10)

,shop_name,name,quantity,price,sum,clear_name,size,brand
2509,ПЕРЕКРЕСТОК,7335 СПм Фарш ДОМАШНИЙ 1кг,1.602,258.99,414.90,спм фарш,1кг,домашний
2207,ПЕРЕКРЕСТОК,5:3472180 Печенье ЮБИЛЕЙНОЕ 116г,1.000,36.90,36.90,печенье,116г,юбилейное
14509,ПЯТЕРОЧКА,3949181 PEPSI Нап.MAX н/кал.б/а с/г 2л,1.000,140.99,140.99,напиток н/кал б/а с/г,2л,pepsi
6372,ПЕРЕКРЕСТОК,4:3659091 Напиток NEMOLOKO 1л,1.000,119.00,119.00,напиток,1л,nemoloko
11224,ПЕРЕКРЕСТОК,*3918530 Салфетки ЧИСТЮЛЯ ЛЕГКАЯ 7шт,1.000,39.99,39.99,салфетки чистюля легкая,7шт,None
263,ПЕРЕКРЕСТОК,6:3644599 Печенье НАДЕЖДА С МАКОМ 1кг,0.314,249.00,78.19,печенье маком,1кг,надежда кф
2831,ПЕРЕКРЕСТОК,*94429 БАЙС.Макар.СПАГЕТ.450г,1.000,30.99,30.99,макар спагет,450г,байсад
3907,ПЕРЕКРЕСТОК,4:44569 Огурцы ЛЮКС фасованные 450г,1.000,99.99,99.99,огурцы люкс фасованные,450г,None
4587,ПЕРЕКРЕСТОК,14*3669585 Филе ЦБ ПЕТРУХА 850г,1.000,199.90,199.90,филе петруха,850г,None
4047,ПЕРЕКРЕСТОК,3501222 Изд.мак.ПРУЖИН 400г,1.000,39.90,39.90,изделие(я) макароны пружин,400г,None


In [95]:
df[['name', 'clear_name', 'size', 'brand', 'quantity', 'price', 'sum']].to_excel('pyat_and_perek.xlsx', index=False)